# Simple Fight 1v1 v102
The current implementation is improved version of [simple-fight-1v1-101](./simple-fight-1v1-101.ipynb) with interactive spreadsheet-like interface.

In [ ]:
%pip -q install ipysheet
%pip -q install pandas

In [12]:

import ipysheet

import pandas as pd
import os
import csv


In [4]:
DEFAULT_UNIT_TYPES = ['soldier', 'knight', 'archer', 'goblin', 'ork', 'axe-thrower']
ATTRIBUTE_NAMES = ['type', 'health', 'attack_damage', 'speed', 'attack_range', 'attack_cooldown']

# we need to check if the local file "simple_fight_1v1.csv" exists
# if not, we need to create it and initialize it with some data
DATA_FILE_NAME = 'work_data/simple_fight_1v1.csv'

def create_clear_data_file(file_name):
    with open(file_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(ATTRIBUTE_NAMES)
        for unit_type in DEFAULT_UNIT_TYPES:
            writer.writerow([unit_type, 100, 10, 10, 10, 1])

if not os.path.exists(DATA_FILE_NAME):
    create_clear_data_file(DATA_FILE_NAME)

# Now we read the data from the file and write down them into pandas DataFrame
df = pd.read_csv(DATA_FILE_NAME, index_col='type')
sheet = ipysheet.from_dataframe(df)
# ipysheet.cell_range(sheet.cells)
def update_data_file(change):
    ipysheet.to_dataframe(sheet).to_csv(DATA_FILE_NAME, index='type', index_label='type')

# Have no idea why but cells are columns in ipysheet and act as cell ranges and we can put observers on them
for cell in sheet.cells:
    cell.observe(update_data_file, 'value')
sheet


Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0[.]0', row_end=5, row_start=0, squeeze_row=Fa…

In [13]:


class UnitAttributes:
    def __init__(self, unit_type: str, unit_attributes: dict):
        self.unit_type = unit_type
        self.attack_damage = unit_attributes['attack_damage']
        self.speed = unit_attributes['speed']
        self.attack_range = unit_attributes['attack_range']
        self.attack_cooldown = unit_attributes['attack_cooldown']
        self.health = unit_attributes['health']

class Unit:
    def __init__(self, unit_type: str, unit_attributes: dict, x=0):
        self.type = unit_type
        self.attributes = UnitAttributes(unit_type, unit_attributes)
        self.max_health = self.attributes.health
        self.current_health = self.max_health
        self.current_attack_cooldown = 0
        self.x = x

DT = 0.01
FIELD_SIZE = 15

def create_unit(unit_type: str, unit_attributes: dict, x=0) -> Unit:
    return Unit(unit_type, unit_attributes, x)

def distance_between(unit1: Unit, unit2: Unit):
    return abs(unit1.x - unit2.x)

def enemy_in_range(unit: Unit, enemy: Unit):
    return distance_between(unit, enemy) <= unit.attributes.attack_range

def unit_can_attack(unit: Unit):
    return unit.current_attack_cooldown <= 0

def process_attack(unit: Unit, enemy: Unit) -> bool:
    if not unit_can_attack(unit):
        return False
    enemy.current_health -= unit.attributes.attack_damage
    unit.current_attack_cooldown = unit.attributes.attack_cooldown
    return True

def move_unit(unit: Unit, direction: int):
    unit.x += unit.attributes.speed * direction * DT

# Unit1 ............................... Unit2
def simulate_fight(unit_type1: str, unit_type2: str, units_attributes: dict):
    unit1 = create_unit(unit_type1, units_attributes[unit_type1], 0)
    unit2 = create_unit(unit_type2, units_attributes[unit_type2], FIELD_SIZE)
    time = 0
    while unit1.current_health > 0 and unit2.current_health > 0:
        if enemy_in_range(unit1, unit2):
            process_attack(unit1, unit2)
        else:
            move_unit(unit1, 1)
        if enemy_in_range(unit2, unit1):
            process_attack(unit2, unit1)
        else:
            move_unit(unit2, -1)
        time += DT
        unit1.current_attack_cooldown = max(0, unit1.current_attack_cooldown - DT)
        unit2.current_attack_cooldown = max(0, unit2.current_attack_cooldown - DT)
    return time, unit1, unit2

def print_results(results):
    for time, unit1, unit2 in results:
        print('-' * 50)
        time = round(time, 1)
        print(f'Fight between {unit1.type} and {unit2.type} lasted {time} seconds')
        if unit1.current_health > 0:
            print(f'Winner: {unit1.type} with remaining health {unit1.current_health}')
        elif unit2.current_health > 0:
            print(f'Winner: {unit2.type} with remaining health {unit2.current_health}')
        else:
            print('Draw, both units died')

# Results are presented as a list of tuples (time, unit1, unit2)
# we want to print it as ASCII table where rows and columns are unit types
# cells are remaining health of unit1 after fight
def print_results_table(results, cell_type="health"):
    # first we need to find all unique unit types
    INTENT = 12
    unit_types = set()
    for _, unit1, unit2 in results:
        unit_types.add(unit1.type)
        unit_types.add(unit2.type)
    unit_types = sorted(list(unit_types))
    # print header
    print(' ' * INTENT, end='')
    for unit_type in unit_types:
        print(f'{unit_type:>{INTENT}}', end='')
    print()
    # print rows
    for unit_type1 in unit_types:
        print(f'{unit_type1:>{INTENT}}', end='')
        for unit_type2 in unit_types:
            # find result for this pair of unit types
            for time, u1, u2 in results:
                if u1.type == unit_type1 and u2.type == unit_type2:
                    break
            else:
                raise Exception(f'No result for {unit_type1} vs {unit_type2}')
            # print cell
            hp = max(u1.current_health, 0)
            if cell_type == "health":
                cell_data = hp
            elif cell_type == "time":
                cell_data = time
            else:
                raise Exception(f'Unknown cell type: {cell_type}')
            if hp == 0:
                print(f'\x1b[31m{cell_data:>{INTENT}.0f}\x1b[0m', end='')
            else:
                print(f'\x1b[32m{cell_data:>{INTENT}.0f}\x1b[0m', end='')
        print()








In [16]:
# Now we need to create a function to get unit attributes from the sheet
unit_types = sheet.row_headers

units_data = {}

for row, unit_type in enumerate(unit_types):
    unit_attributes = {}
    for col, attribute in enumerate(ATTRIBUTE_NAMES[1:]):
        unit_attributes[attribute] = sheet.cells[col].value[row]
    unit_attributes["type"] = unit_type
    units_data[unit_type] = unit_attributes

units_data

{'soldier': {'health': 100,
  'attack_damage': 10,
  'speed': 10,
  'attack_range': 10,
  'attack_cooldown': 1,
  'type': 'soldier'},
 'knight': {'health': 100,
  'attack_damage': 10,
  'speed': 10,
  'attack_range': 10,
  'attack_cooldown': 1,
  'type': 'knight'},
 'archer': {'health': 100,
  'attack_damage': 10,
  'speed': 10,
  'attack_range': 10,
  'attack_cooldown': 1,
  'type': 'archer'},
 'goblin': {'health': 100,
  'attack_damage': 10,
  'speed': 10,
  'attack_range': 10,
  'attack_cooldown': 1,
  'type': 'goblin'},
 'ork': {'health': 100,
  'attack_damage': 10,
  'speed': 10,
  'attack_range': 10,
  'attack_cooldown': 1,
  'type': 'ork'},
 'axe-thrower': {'health': 100,
  'attack_damage': 10,
  'speed': 10,
  'attack_range': 10,
  'attack_cooldown': 1,
  'type': 'axe-thrower'}}

In [17]:
results = []
unit_types = units_data.keys()  

for unit_type1 in unit_types:
    for unit_type2 in unit_types:
        t, u1, u2 = simulate_fight(unit_type1, unit_type2, units_data)
        results.append((t, u1, u2))

print("-" * 40 + " Health: " + "-" * 38)
print_results_table(results, cell_type="health")
print("-" * 40 + " Time: " + "-" * 40)
print_results_table(results, cell_type="time")


---------------------------------------- Health: --------------------------------------
                  archer axe-thrower      goblin      knight         ork     soldier
      archer           0           0           0           0           0           0
 axe-thrower           0           0           0           0           0           0
      goblin           0           0           0           0           0           0
      knight           0           0           0           0           0           0
         ork           0           0           0           0           0           0
     soldier           0           0           0           0           0           0
---------------------------------------- Time: ----------------------------------------
                  archer axe-thrower      goblin      knight         ork     soldier
      archer           9           9           9           9           9           9
 axe-thrower           9           9           9           